# Transfer Learning

- uses pre existing knowledge by reusing models trained on large and comprehensive datasets

- enables knowledge gained from one task and apply it to a different but related task

- speed up training and improves performance with limited data

- Fine-tuning pre-trained model trained on a large dataset like ImageNet

- features from original training transferred to a new task



# Implementing Transfer Learning with Keras

In [1]:
!pip install --upgrade tensorflow numpy pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 771.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 42.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.2.5 requires numpy<2.0.0,>=1.19.0; python_version >= "3.9", but you have numpy 2.0.2 which is incompatible.
pytensor 2.27.1 requires numpy<2,>=1.17.0, but you have numpy 2.0.2 which is incompatible.
langchain 0.3.18 requires numpy<2,>=1.26.4; python_version < "3.12", but you have numpy 2.0.2 which is incompatible.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.0.2 which is incompatible.


In [1]:
!pip install numpy==1.26.4

In [2]:
import sys
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Input, Model
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense
from PIL import Image
import numpy as np

In [ ]:
# Increase recursion limit
sys.setrecursionlimit(10000)

In [ ]:
# Ensure the dataset directory exists and generate sample data if needed
def generate_sample_data():
  os.makedirs('/content/training_data/class_1', exist_ok=True)
  os.makedirs('/content/training_data/class_2', exist_ok=True)

  for i in range(10):
    img = Image.fromarray(np.random.randint(0, 255, size=(224, 224, 3), dtype=np.uint8))
    img.save(f'/content/training_data/class_1/image_{i}.jpg')
    img.save(f'/content/training_data/class_2/image_{i}.jpg')

In [ ]:
# Load the VGG16 model pre-trained on ImageNet
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [ ]:
# Freeze the base model layers
for layer in base_model.layers:
  layer.trainable = False

In [ ]:
# create a new model and add the base model and new layers
model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Load and preprocess the dataset
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    'training_data',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

In [ ]:
model.fit(train_generator, epochs=10)